In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf

In [2]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)


[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]


In [4]:
#column="Close"

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
dataset=data1

In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [26]:
def combination(dataset, listt):
    """
    Function to perform Simple Exponential Smoothing (SES) algorithm for forecasting.

    Parameters:
    - dataset: Pandas DataFrame or Series containing the time series data.
    - listt: List of variables to use for forecasting.

    Returns:
    - perf: DataFrame containing performance metrics.
    - model: Fitted SES model.
    - pred: Forecasted values.
    """
    print(listt)
    datasetTwo = dataset[listt]
    test_obs = 28
    train = datasetTwo[:-test_obs][listt[0]]  # Selecting the first variable for training
    test = datasetTwo[-test_obs:]

    from statsmodels.tsa.holtwinters import SimpleExpSmoothing
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

    performance = {
        "Model": [],
        "RMSE": [],
        "MaPe": [],
        "Test": []
    }

    best_alpha = None
    best_rmse = float('inf')

    for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:  # Try different alpha values
        model = SimpleExpSmoothing(train)
        fit_model = model.fit(smoothing_level=alpha)

        # Forecast
        pred = fit_model.forecast(len(test))  # Forecast for the same number of observations as test data

        # Calculate RMSE
        rmse = mean_squared_error(test[listt[0]], pred, squared=False)  # Selecting the first variable for comparison

        # Check if current model has lower RMSE
        if rmse < best_rmse:
            best_rmse = rmse
            best_alpha = alpha
            best_pred = pred

    print("Best alpha:", best_alpha)
    print("Best RMSE:", best_rmse)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(test[listt[0]], best_pred)  # Selecting the first variable for comparison

    # Store performance metrics
    performance["Model"].append(listt)
    performance["RMSE"].append(best_rmse)
    performance["MaPe"].append(mape)
    performance["Test"].append(test_obs)

    perf = pd.DataFrame(performance)
    return perf, fit_model, best_pred


In [27]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [28]:
perf,result,pred=combination(data1, listt)

['Close', 'High', 'Open', 'Low']
Best alpha: 0.4
Best RMSE: 0.1485861963993438


C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\MANIKA\Anaconda3\envs\iml\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model ha

In [29]:
data1

,Open,High,Low,Close
0,0.508982,0.441549,0.543815,0.491495
1,0.497006,0.433854,0.540997,0.513145
2,0.544411,0.479275,0.593970,0.563144
3,0.543912,0.504095,0.604114,0.552835
4,0.538423,0.489203,0.587489,0.576289
...,...,...,...,...
138,0.385230,0.330107,0.398422,0.404639
139,0.414421,0.355920,0.417864,0.401031
140,0.369261,0.296848,0.338405,0.316495
141,0.334331,0.252916,0.226543,0.275773


In [30]:
perf

,Model,RMSE,MaPe,Test
0,"[Close, High, Open, Low]",0.148586,0.809591,28


In [31]:
pred

115    0.28939
116    0.28939
117    0.28939
118    0.28939
119    0.28939
120    0.28939
121    0.28939
122    0.28939
123    0.28939
124    0.28939
125    0.28939
126    0.28939
127    0.28939
128    0.28939
129    0.28939
130    0.28939
131    0.28939
132    0.28939
133    0.28939
134    0.28939
135    0.28939
136    0.28939
137    0.28939
138    0.28939
139    0.28939
140    0.28939
141    0.28939
142    0.28939
dtype: float64